In [ ]:
# CONTEXTO

# Vamos a ver si podemos mejorar los resultados del modelo LGBM único
# Para ello haremos un stacking, pero no en paralelo, sino en segmentado
# Usaremos el criterio de negocio para seleccionar las variables más significativas para la clusterización
# Usaremos KPrototypes, que acepta variables numéricas y categóricas
# Podríamos usar las importancias del modelo lightGBM geenral, pero éste se entrenó con la variable objetivo, por lo que estaríamos metiendo sesgo y/o información futura a los neuvos datos

#### Preparativos

In [2]:
# Instalaciones
!pip install lightgbm

In [3]:
!pip install kmodes

In [29]:
# Importaciones

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier

from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

In [6]:
# Deslimitar/Limitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

#### Preparación Dataset

In [7]:
# Read Matches data

df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')

C:\Users\Diego\AppData\Local\Temp\ipykernel_6908\2151874057.py:3: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')


In [8]:
df0.head()

,ID_RESERVA,ID_HOTEL,HOTEL,LLEGADA,LLEGADA_ANO,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA,SALIDA_ANO,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,ID_TIPO,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,ID_CLIENTE,TIPO_CLIENTE,CLIENTE,GRUPO,ID_MONEDA,MONEDA,STATUS,MOTIVO,CHECKIN,SUPLETORIA,CUNAS,FECHA_TOMA,FECHA_TOMA_ANO,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,FECHA_MOD,FECHA_MOD_ANO,FECHA_MOD_MES,FECHA_MOD_DIAm,FECHA_MOD_DIAs,FECHA_MOD_AVANCE,FECHA_CANCELACION,FECHA_CANCELACION_ANO,FECHA_CANCELACION_MES,FECHA_CANCELACION_DIAm,FECHA_CANCELACION_DIAs,FECHA_CANCELACION_AVANCE,LT_TOMA_LLEGADA,LT_TOMA_CANCELACION,ID_FIDELIDAD,FIDELIDAD,VALHAB,VALPEN,VALSERV,VALFIJOS,COMERCIALIZADORA,CMVALHAB,CMVALPEN,CMCVALSERV,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,AUTORIZO,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,CANCELADA
0,456094009201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-12-28,2022,12,28,3,0.989041095890411,2023-01-04,2023,1,4,3,0.00821917808219178,7,Media,AllInclusive,23300,LHAmbassadorSuiteBSOV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,1,DESCONOCIDO_0,1,0,0,2022-01-13,2022,1,13,4,0.03287671232876712,2022-12-28,2022,12.0,28.0,3.0,0.989041095890411,NaN,NaT,NaN,NaN,NaN,NaN,349,NaN,0,Ninguna,1868.0,708.0,0.0,0.0,0,1868.0,708.0,0.0,2576.0,858.6666666666666,368.0,122.66666666666666,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,0
1,425835909201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-01-22,2022,1,22,6,0.057534246575342465,2022-01-29,2022,1,29,6,0.07671232876712329,7,Media,AllInclusive,43800,LHJuniorSuiteGV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2021-07-13,2021,7,13,2,0.5287671232876713,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,193,191.0,0,Ninguna,1329.3,665.0,0.0,0.0,0,1329.3,665.0,0.0,1994.3,664.7666666666667,284.9,94.96666666666667,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
2,549929510601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-14,2023,11,14,2,0.8684931506849315,8,Media,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-04-17,2023,4,17,1,0.29041095890410956,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-10-17,2023,10.0,17.0,2.0,0.7917808219178082,203,183.0,0,Ninguna,1146.56,1146.56,0.0,12.48,0,1146.56,1146.56,0.0,2305.6,1152.8,288.2,144.1,1,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,1
3,519229110601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-23,2023,11,23,4,0.8931506849315068,17,Larga,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-22,2023,2,22,3,0.14246575342465753,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-09-14,2023,9.0,14.0,4.0,0.7013698630136986,257,204.0,0,Ninguna,2436.44,2436.44,0.0,26.52,0,2436.44,2436.44,0.0,4899.400000000001,2449.7000000000003,288.20000000000005,144.10000000000002,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
4,537405910601,106,ComplejoRivieraMaya,2023-11-25,2023,11,25,6,0.8986301369863013,2023-12-09,2023,12,9,6,0.936986301369863,14,Larga,AllInclusive,13000,COLJuniorSuiteGV,1,1,Single,1,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-17,2023,2,17,5,0.12876712328767123,2023-11-26,2023,11.0,26.0,7.0,0.9013698630136986,2023-04-11,2023,4.0,11.0,2.0,0.273972602739726,281,53.0,0,Ninguna,2413.18,0.0,0.0,21.84,0,2413.18,0.0,0.0,2435.02,2435.02,173.93,173.93,1,0,RUMANIA,EURESTE,Fixedrates,DIRECTSALES,1


In [9]:
df0.columns

Index(['ID_RESERVA', 'ID_HOTEL', 'HOTEL', 'LLEGADA', 'LLEGADA_ANO',
       'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA', 'SALIDA_ANO', 'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs',
       'SALIDA_AVANCE', 'NOCHES', 'DURACION_ESTANCIA', 'REGIMEN', 'ID_TIPO',
       'TIPO', 'USO', 'PAX_NUM', 'PAX_CAT', 'ADULTOS', 'NENES', 'BEBES',
       'ID_CLIENTE', 'TIPO_CLIENTE', 'CLIENTE', 'GRUPO', 'ID_MONEDA', 'MONEDA',
       'STATUS', 'MOTIVO', 'CHECKIN', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA',
       'FECHA_TOMA_ANO', 'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm',
       'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE', 'FECHA_MOD', 'FECHA_MOD_ANO',
       'FECHA_MOD_MES', 'FECHA_MOD_DIAm', 'FECHA_MOD_DIAs', 'FECHA_MOD_AVANCE',
       'FECHA_CANCELACION', 'FECHA_CANCELACION_ANO', 'FECHA_CANCELACION_MES',
       'FECHA_CANCELACION_DIAm', 'FECHA_CANCELACION_DIAs',
       'FECHA_CANCELACION_AVANCE', 'LT_TOMA_LLEGADA', 'LT_TOMA_CANCELACION',
       'ID_FIDELIDAD', 'FIDELIDAD', 'VALHAB', '

#### Preparación de un DF Mixto

In [11]:
df_mix = df0.copy()

In [12]:
# Reconfigurar dtypes

df_mix.drop(columns=['ID_RESERVA'], inplace=True) # No usar, ID
df_mix.drop(columns=['ID_HOTEL'], inplace=True) # No usar, ID
df_mix['HOTEL'] = df_mix['HOTEL'].astype('category', errors='raise')
df_mix.drop(columns=['LLEGADA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['LLEGADA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['LLEGADA_MES'] = pd.to_numeric(df_mix['LLEGADA_MES'], errors='raise').astype('category')
df_mix['LLEGADA_DIAm'] = pd.to_numeric(df_mix['LLEGADA_DIAm'], errors='raise').astype('category')
df_mix['LLEGADA_DIAs'] = pd.to_numeric(df_mix['LLEGADA_DIAs'], errors='raise').astype('category')
df_mix['LLEGADA_AVANCE'] = pd.to_numeric(df_mix['LLEGADA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['SALIDA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['SALIDA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['SALIDA_MES'] = pd.to_numeric(df_mix['SALIDA_MES'], errors='raise').astype('category')
df_mix['SALIDA_DIAm'] = pd.to_numeric(df_mix['SALIDA_DIAm'], errors='raise').astype('category')
df_mix['SALIDA_DIAs'] = pd.to_numeric(df_mix['SALIDA_DIAs'], errors='raise').astype('category')
df_mix['SALIDA_AVANCE'] = pd.to_numeric(df_mix['SALIDA_AVANCE'], errors='raise').astype(float)
df_mix['NOCHES'] = pd.to_numeric(df_mix['NOCHES'], errors='raise').astype('Int64')
df_mix['DURACION_ESTANCIA'] = df_mix['DURACION_ESTANCIA'].astype('category', errors='raise')
df_mix['REGIMEN'] = df_mix['REGIMEN'].astype('category', errors='raise')
df_mix.drop(columns=['ID_TIPO'], inplace=True) # No usar, ID
df_mix['TIPO'] = df_mix['TIPO'].astype('category', errors='raise')
df_mix['USO'] = pd.to_numeric(df_mix['USO'], errors='raise').astype('Int64')
df_mix['PAX_NUM'] = pd.to_numeric(df_mix['PAX_NUM'], errors='raise').astype('Int64')
df_mix['PAX_CAT'] = df_mix['PAX_CAT'].astype('category', errors='raise')
df_mix['ADULTOS'] = pd.to_numeric(df_mix['ADULTOS'], errors='raise').astype('Int64')
df_mix['NENES'] = pd.to_numeric(df_mix['NENES'], errors='raise').astype('Int64')
df_mix['BEBES'] = pd.to_numeric(df_mix['BEBES'], errors='raise').astype('Int64')
df_mix.drop(columns=['ID_CLIENTE'], inplace=True) # No usar, ID
df_mix['TIPO_CLIENTE'] = pd.to_numeric(df_mix['TIPO_CLIENTE'], errors='raise').astype('category')
df_mix['CLIENTE'] = df_mix['CLIENTE'].astype('category', errors='raise')
df_mix['GRUPO'] = pd.to_numeric(df_mix['GRUPO'], errors='raise').astype('category')
df_mix.drop(columns=['ID_MONEDA'], inplace=True) # No usar, ID
df_mix['MONEDA'] = df_mix['MONEDA'].astype('category', errors='raise')
df_mix.drop(columns=['STATUS'], inplace=True) # No usar, redundante con variable objetivo
df_mix.drop(columns=['MOTIVO'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['CHECKIN'], inplace=True) # No usar, imposible ver el futuro
df_mix['SUPLETORIA'] = pd.to_numeric(df_mix['SUPLETORIA'], errors='raise').astype('Int64')
df_mix['CUNAS'] = pd.to_numeric(df_mix['CUNAS'], errors='raise').astype('Int64')
df_mix.drop(columns=['FECHA_TOMA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['FECHA_TOMA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['FECHA_TOMA_MES'] = pd.to_numeric(df_mix['FECHA_TOMA_MES'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAm'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAm'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAs'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAs'], errors='raise').astype('category')
df_mix['FECHA_TOMA_AVANCE'] = pd.to_numeric(df_mix['FECHA_TOMA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['FECHA_MOD'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix['LT_TOMA_LLEGADA'] = pd.to_numeric(df_mix['LT_TOMA_LLEGADA'], errors='raise').astype('Int64')
df_mix.drop(columns=['LT_TOMA_CANCELACION'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['ID_FIDELIDAD'], inplace=True) # No usar, ID
df_mix['FIDELIDAD'] = df_mix['FIDELIDAD'].astype('category', errors='raise')
df_mix.drop(columns=['VALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALFIJOS'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['COMERCIALIZADORA'] = pd.to_numeric(df_mix['COMERCIALIZADORA'], errors='raise').astype('category')
df_mix.drop(columns=['CMVALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMVALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMCVALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['VALOR_USD'] = pd.to_numeric(df_mix['VALOR_USD'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX'] = pd.to_numeric(df_mix['VALOR_USD_PAX'], errors='raise').astype(float)
df_mix['VALOR_USD_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_NOCHE'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_PAX_NOCHE'], errors='raise').astype(float)
df_mix.drop(columns=['AUTORIZO'], inplace=True) # No usar, de momento no muy claro
df_mix['GRATIS'] = pd.to_numeric(df_mix['GRATIS'], errors='raise').astype('category')
df_mix['PAIS'] = df_mix['PAIS'].astype('category', errors='raise')
df_mix['CONTINENTE'] = df_mix['CONTINENTE'].astype('category', errors='raise')
df_mix['SEGMENTO'] = df_mix['SEGMENTO'].astype('category', errors='raise')
df_mix['FUENTE_NEGOCIO'] = df_mix['FUENTE_NEGOCIO'].astype('category', errors='raise')
df_mix['CANCELADA'] = pd.to_numeric(df_mix['CANCELADA'], errors='raise').astype('category')

In [13]:
df_mix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   HOTEL                1139289 non-null  category
 1   LLEGADA_MES          1139289 non-null  category
 2   LLEGADA_DIAm         1139289 non-null  category
 3   LLEGADA_DIAs         1139289 non-null  category
 4   LLEGADA_AVANCE       1139289 non-null  float64 
 5   SALIDA_MES           1139289 non-null  category
 6   SALIDA_DIAm          1139289 non-null  category
 7   SALIDA_DIAs          1139289 non-null  category
 8   SALIDA_AVANCE        1139289 non-null  float64 
 9   NOCHES               1139289 non-null  Int64   
 10  DURACION_ESTANCIA    1139289 non-null  category
 11  REGIMEN              1139289 non-null  category
 12  TIPO                 1139289 non-null  category
 13  USO                  1139289 non-null  Int64   
 14  PAX_NUM              1139289 non-n

In [14]:
X_mix = df_mix.drop(columns=['CANCELADA'])

y_mix = df_mix['CANCELADA']

X_mix_train, X_mix_test, y_mix_train, y_mix_test = train_test_split(X_mix, y_mix, test_size=0.2, random_state=42)

# El test no lo tocaremos nunca hasta el final

#### Clusterización no supervisada (Prueba)

In [ ]:
df_kproto = X_mix.copy()

# Descomentamos si el entrenamiento se nos alarga demasiado
# Cortamos el DF lo suficientemente grande como para no perder demasiada información
#df_kproto = df_kproto.sample(n=100000, random_state=42)

In [ ]:
# Selección de 10 columnas para clusterizar
# Con las categóricas debemos cuidarnos de que los datos nuevos no contengancategorías no contempladas aquí
# Usando un 80% del dataset en train, no deberíamos tener problema

df_kproto = df_kproto[[
    'HOTEL',
    #'LLEGADA_MES',
    #'LLEGADA_DIAm',
    #'LLEGADA_DIAs',
    #'LLEGADA_AVANCE',
    #'SALIDA_MES',
    #'SALIDA_DIAm',
    #'SALIDA_DIAs',
    #'SALIDA_AVANCE',
    #'NOCHES',
    'DURACION_ESTANCIA',
    'REGIMEN',
    'TIPO',
    #'USO',
    #'PAX_NUM',
    'PAX_CAT',
    #'ADULTOS',
    #'NENES',
    #'BEBES',
    #'TIPO_CLIENTE',
    'CLIENTE',
    #'GRUPO',
    'MONEDA',
    #'SUPLETORIA',
    #'CUNAS',
    #'FECHA_TOMA_MES',
    #'FECHA_TOMA_DIAm',
    #'FECHA_TOMA_DIAs',
    #'FECHA_TOMA_AVANCE',
    #'LT_TOMA_LLEGADA',
    'FIDELIDAD',
    'COMERCIALIZADORA',
    #'VALOR_USD',
    #'VALOR_USD_PAX',
    #'VALOR_USD_NOCHE',
    #'VALOR_USD_PAX_NOCHE',
    #'GRATIS',
    #'PAIS',
    'CONTINENTE',
    #'SEGMENTO',
    #'FUENTE_NEGOCIO',
]]

In [20]:
# Tipos de variables
num_cols = df_kproto.select_dtypes(include='number').columns
cat_cols = df_kproto.select_dtypes(include='category').columns

In [27]:
# Estandarizamos numéricas, comentamos porque en este caso no hay
#df_kproto[num_cols] = StandardScaler().fit_transform(df_kproto[num_cols])

In [22]:
df_kproto.head()

,HOTEL,DURACION_ESTANCIA,REGIMEN,TIPO,PAX_CAT,CLIENTE,MONEDA,FIDELIDAD,COMERCIALIZADORA,CONTINENTE
0,GrandPalladiumJamaica&LadyHamiltonResort,Media,AllInclusive,LHAmbassadorSuiteBSOV,Familias,PALLADIUMTRAVELCLUB_SOCIOS,USD,Ninguna,0,AMERICA
1,GrandPalladiumJamaica&LadyHamiltonResort,Media,AllInclusive,LHJuniorSuiteGV,Familias,PALLADIUMTRAVELCLUB_SOCIOS,USD,Ninguna,0,AMERICA
2,ComplejoRivieraMaya,Media,AllInclusive,TRSJuniorSuitePS,Parejas,PALLADIUMTRAVELCLUB_SOCIOS,USD,Ninguna,0,AMERICA
3,ComplejoRivieraMaya,Larga,AllInclusive,TRSJuniorSuitePS,Parejas,PALLADIUMTRAVELCLUB_SOCIOS,USD,Ninguna,0,AMERICA
4,ComplejoRivieraMaya,Larga,AllInclusive,COLJuniorSuiteGV,Single,PALLADIUMTRAVELCLUB_SOCIOS,USD,Ninguna,0,EURESTE


In [23]:
# El algoritmo que utilizaremos necesitará los índices. Así que:
cat_idx = [df_mix.columns.get_loc(col) for col in cat_cols]

In [24]:
# Clusterizador

# cat_cols a string para que el algoritmo no se queje
df_kproto[cat_cols] = df_kproto[cat_cols].astype(str)

# Convertimos a matriz numpy
matriz_kproto = df_kproto.to_numpy()

C:\Users\Diego\AppData\Local\Temp\ipykernel_6908\2666638428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kproto[cat_cols] = df_kproto[cat_cols].astype(str)


In [28]:
# Entrena el modelo
#kproto = KPrototypes(n_clusters=3, init='Cao', n_init=1, verbose=1, random_state=42)
#clusters = kproto.fit_predict(matriz_kproto, categorical=cat_idx)

# Añade los clusters al DataFrame
#df_kproto['cluster'] = clusters

In [ ]:
# Como nuestra elección se basa únicamente en datos categóricos, nos arroja este error:
# AssertionError: All columns are categorical, use k-modes instead of k-prototypes.

In [31]:
kmodes = KModes(n_clusters=3, init='Cao', n_init=1, verbose=2, random_state=42)

# Fit + predict
clusters = kmodes.fit_predict(matriz_kproto)

# Añadir clusters al df
df_kproto['cluster'] = clusters

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 265034, cost: 4041366.0


In [32]:
# Ojo, vamos a ver si los clústeres estarían balanceados, aunque fuera mínimamente
df_kproto['cluster'].value_counts()

cluster
0    657745
1    311285
2    170259
Name: count, dtype: int64

In [33]:
# Algo desbalanceado pero el grupo más pequeño tiene más que suficientes líneas para un entrenamiento decente
# Concatenamos al df de entrenamiento
X_mix_train['cluster'] = df_kproto['cluster']

In [35]:
X_mix_train.head(15)

,HOTEL,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,TIPO_CLIENTE,CLIENTE,GRUPO,MONEDA,SUPLETORIA,CUNAS,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,LT_TOMA_LLEGADA,FIDELIDAD,COMERCIALIZADORA,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,cluster
735208,ComplejoRivieraMaya,7,9,7,0.517808,7,21,5,0.550685,12,Media,AllInclusive,WSJuniorSuiteGV,1,2,Parejas,1,1,0,1,DELFOSOPERADORMAYORISTASRL,0,USD,0,0,4,18,2,0.293151,82,Ninguna,1,3067.20,1533.600000,255.600000,127.800000,0,ARGENTINA,AMERICA,FixedRates,T.O./T.A.,1
966327,GrandPalladiumJamaica&LadyHamiltonResort,5,9,1,0.350685,5,16,1,0.369863,7,Media,AllInclusive,LHJuniorSuiteBSOV,2,2,Parejas,2,0,0,1,PALLADIUMTRAVELCLUB_SOCIOS,0,USD,0,0,4,19,2,0.295890,20,Ninguna,0,2149.84,1074.920000,307.120000,153.560000,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,0
120200,ComplejoPuntaCana,12,12,2,0.945205,12,14,4,0.950685,2,Corta,AllInclusive,BAVSuperiorJSGV,2,2,Parejas,2,0,0,1,MILENATOURS,1,USD,0,0,12,6,3,0.928767,6,Ninguna,0,440.00,220.000000,220.000000,110.000000,0,DOMINICA,AMERICA,GroupLeisure,T.O./T.A.,0
177314,ComplejoPuntaCana,2,11,5,0.112329,2,15,2,0.123288,4,Corta,AllInclusive,PALDeluxeGardenView,2,2,Parejas,2,0,0,1,HOTELCOLLECT_EXPEDIA,0,USD,0,0,12,29,3,0.991781,44,Ninguna,0,1333.20,666.600000,333.300000,166.650000,0,ESTADOSUNIDOS,AMERICA,BAR,E-COMMERCE,0
553733,ComplejoRivieraMaya,8,17,4,0.624658,8,25,5,0.646575,8,Media,AllInclusive,COLDeluxeGardenView,2,2,Parejas,2,0,0,1,AMERICANEXECUTIVEINTERNATIONAL,0,USD,0,0,7,26,3,0.564384,22,Ninguna,1,1414.60,707.300000,176.825000,88.412500,0,URUGUAY,AMERICA,FixedRates,T.O./T.A.,1
1057923,ComplejoRivieraMaya,8,27,6,0.652055,8,28,7,0.654795,1,Corta,AllInclusive,COLJuniorSuiteGV,2,2,Parejas,2,0,0,1,BOOKINGEUROPEBV,0,USD,0,0,8,25,4,0.646575,2,Ninguna,0,450.90,225.450000,450.900000,225.450000,0,ESPANA,EUROPA,BAR,E-COMMERCE,0
551228,ComplejoPuntaCana,5,7,6,0.345205,5,11,3,0.356164,4,Corta,AllInclusive,BAVRooftopJTJS,1,2,Parejas,1,1,1,1,ROIBACK(GLOBALOBIS.L.),0,USD,0,1,1,26,3,0.068493,101,PalladiumRewards,0,830.12,415.060000,207.530000,103.765000,0,ESTADOSUNIDOS,AMERICA,Loyalty,DIRECTSALES,0
314790,GrandPalladiumJamaica&LadyHamiltonResort,6,14,3,0.449315,6,19,1,0.463014,5,Corta,AllInclusive,JAMJuniorSuiteGV,2,3,Familias,3,0,0,1,TRAVELZAPINC.USA,0,USD,0,0,1,10,2,0.024658,155,Ninguna,1,2081.30,693.766667,416.260000,138.753333,0,ESTADOSUNIDOS,AMERICA,FixedRates,T.O./T.A.,2
284829,GrandPalladiumJamaica&LadyHamiltonResort,9,30,6,0.745205,10,3,2,0.753425,3,Corta,AllInclusive,LHJuniorSuiteBS,2,2,Parejas,2,0,0,1,EXPEDIA,0,USD,0,0,8,29,2,0.657534,32,Ninguna,1,633.48,316.740000,211.160000,105.580000,0,ESTADOSUNIDOS,AMERICA,BAR,E-COMMERCE,2
326758,ComplejoPuntaCana,9,29,5,0.742466,10,6,5,0.761644,7,Media,AllInclusive,BAVSuperiorJSGV,2,2,Parejas,2,0,0,1,SUNWINGTRAVELGROUPINC.,0,USD,0,0,8,17,3,0.624658,408,Ninguna,1,1043.00,521.500000,149.000000,74.500000,0,CANADA,AMERICA,FixedRates,T.O./T.A.,1
